In [1]:
import pickle

In [2]:
PATH='/home/kirana/Documents/phd/experiment1_new/DNNREC_ML100k'
DATAPATH='/home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-latest-small'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import sklearn as sklearn
import datetime as dt 
import seaborn as sns
import torch as torch

In [4]:
[dftrain,dfvalid]=pickle.load(open(f'{DATAPATH}/df.pkl','rb'))

In [5]:
dftrain.shape, dfvalid.shape

((90543, 14), (10293, 14))

In [6]:
dftrain.columns

Index(['flag_train', 'genres', 'imdbId', 'itemId', 'maxSeqNum', 'rating',
       'seqNum', 'tag', 'timestamp', 'title', 'tmdbId', 'userId', 'year',
       'year_acq'],
      dtype='object')

In [7]:
item_features=['tag','title','genres']

In [8]:
dftrain[item_features].head()

,tag,title,genres
0,NaN,She's the One (1996),Comedy|Romance
1,NaN,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi
2,NaN,Bambi (1942),Animation|Children|Drama
3,NaN,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Sci-Fi
4,NaN,"13th Warrior, The (1999)",Action|Adventure|Fantasy


In [9]:
dftrain['genres']=[item.replace("|", " ") for item in dftrain['genres']]
dfvalid['genres']=[item.replace("|", " ") for item in dfvalid['genres']]

In [10]:
from sklearn import feature_extraction, preprocessing
myvec_genre=sklearn.feature_extraction.text.CountVectorizer (ngram_range=(1,1),lowercase=True)
one_hot_genre_train=myvec_genre.fit_transform(dftrain['genres'])
one_hot_genre_valid=myvec_genre.transform(dfvalid['genres'])

In [11]:
from sklearn import feature_extraction, preprocessing
myvec_title=sklearn.feature_extraction.text.CountVectorizer (ngram_range=(1,1),lowercase=True,max_features=250)
one_hot_title_train=myvec_title.fit_transform(dftrain['title'])
one_hot_title_valid=myvec_title.transform(dfvalid['title'])

In [12]:
from sklearn import feature_extraction, preprocessing
myvec_tag=sklearn.feature_extraction.text.CountVectorizer (ngram_range=(1,1),lowercase=True,max_features=100)
one_hot_tag_train=myvec_tag.fit_transform(dftrain['title'])
one_hot_tag_valid=myvec_tag.transform(dfvalid['title'])

In [13]:
one_hot_title_train.shape, one_hot_genre_train.shape,one_hot_tag_train.shape

((90543, 250), (90543, 24), (90543, 100))

In [14]:
one_hot_title_train

<90543x250 sparse matrix of type '<class 'numpy.int64'>'
	with 201719 stored elements in Compressed Sparse Row format>

In [15]:
df_sideinfo_title_train=pd.DataFrame(one_hot_title_train.todense(),columns=['f_title_'+x for x in myvec_title.get_feature_names()])
df_sideinfo_genre_train=pd.DataFrame(one_hot_genre_train.todense(),columns=['f_genre_'+x for x in myvec_genre.get_feature_names()])
df_sideinfo_tag_train=pd.DataFrame(one_hot_tag_train.todense(),columns=['f_tag_'+x for x in myvec_tag.get_feature_names()])

In [16]:
df_sideinfo_title_valid=pd.DataFrame(one_hot_title_valid.todense(),columns=['f_title_'+x for x in myvec_title.get_feature_names()])
df_sideinfo_genre_valid=pd.DataFrame(one_hot_genre_valid.todense(),columns=['f_genre_'+x for x in myvec_genre.get_feature_names()])
df_sideinfo_tag_valid=pd.DataFrame(one_hot_tag_valid.todense(),columns=['f_tag_'+x for x in myvec_tag.get_feature_names()])

In [17]:
df_sideinfo_title_train.shape,df_sideinfo_genre_train.shape,df_sideinfo_tag_train.shape

((90543, 250), (90543, 24), (90543, 100))

In [18]:
df_sideinfo_title_valid.shape,df_sideinfo_genre_valid.shape,df_sideinfo_tag_valid.shape

((10293, 250), (10293, 24), (10293, 100))

In [19]:
dftrain.shape

(90543, 14)

In [20]:
dftrain.dtypes

flag_train             int64
genres                object
imdbId                 int64
itemId                 int64
maxSeqNum            float64
rating               float64
seqNum                 int64
tag                   object
timestamp     datetime64[ns]
title                 object
tmdbId               float64
userId                 int64
year                   int64
year_acq               int64
dtype: object

In [21]:
dftrain.shape,df_sideinfo_title_train.shape,df_sideinfo_genre_train.shape,df_sideinfo_tag_train.shape

((90543, 14), (90543, 250), (90543, 24), (90543, 100))

In [22]:
dftrain.reset_index(inplace=True)
dfvalid.reset_index(inplace=True)

In [23]:
dftrain=pd.concat([dftrain,df_sideinfo_title_train,df_sideinfo_genre_train,df_sideinfo_tag_train],axis=1)

In [24]:
dfvalid=pd.concat([dfvalid,df_sideinfo_title_valid,df_sideinfo_genre_valid,df_sideinfo_tag_valid],axis=1)

In [25]:
dftrain.shape,dfvalid.shape

((90543, 389), (10293, 389))

In [26]:
dftrain.dtypes

index                    int64
flag_train               int64
genres                  object
imdbId                   int64
itemId                   int64
maxSeqNum              float64
rating                 float64
seqNum                   int64
tag                     object
timestamp       datetime64[ns]
title                   object
tmdbId                 float64
userId                   int64
year                     int64
year_acq                 int64
f_title_12               int64
f_title_13               int64
f_title_1940             int64
f_title_1954             int64
f_title_1959             int64
f_title_1960             int64
f_title_1962             int64
f_title_1963             int64
f_title_1964             int64
f_title_1967             int64
f_title_1968             int64
f_title_1970             int64
f_title_1971             int64
f_title_1972             int64
f_title_1973             int64
                     ...      
f_tag_life               int64
f_tag_li

In [27]:
pickle.dump([dftrain,dfvalid],open(f'{DATAPATH}/df_side.pkl','wb'))